## WICCI Downscaled Data: Processing and aggregating climate scenario -- Tmin for all years -- 10 year intervals ##

Objectives
* aggregate data for tmin, e.g. mean or standard deviation, for RCP4.5 and RCP8.5
* run for every 10 year model window, e.g. 2005-2015 for 2010 average
* creat new netcdf file(s) for aggregate data for each 10-year timeframe

Eric Compas, compase@uww.edu 11/17/2021

In [1]:
import netCDF4
import numpy as np
import os

In [2]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [3]:
#base_folder = "Z:/Climate_Data"
base_folder = "F:/Climate_Data"
if not os.path.isdir(base_folder):
    print("Base folder not valid")

In [4]:
#out_folder = "Z:/Climate_Data/aggregate_files"
out_folder = "C:/Users/Eric/Dropbox/Climate_Data/aggregate_files"
if not os.path.isdir(base_folder):
    print("Out folder not valid")

The BIG LOOP.

Look across all years, models, and create aggregate file for each year.

In [ ]:
# loop across models
models = ["rcp45","rcp85"]
for m in models:
    
    # get subfolders for current model
    folders = os.listdir(os.path.join(base_folder,m))
    
    # loop across years in 10-year increments
    for y in range(2010,2101,10):
        print("Processing base year "+str(y))
        
        # get mean, min, max, and std for tmin for current 10-year window
        tmin_array = []
        for yr in range(y-5,y+5):
            for f in folders:
                realizations = ["01","02","03"]
                for r in realizations:
                    nf = os.path.join(base_folder,m,f,"r1i1p1","temp_"+r+"_"+str(yr)+".nc")
                    #print("Adding "+nf)
                    try:
                        n = netCDF4.Dataset(nf)
                    except:
                        #print("File not found: "+nf)
                        q = 0
                    else:
                        tmin = n.variables['tmin']
                        # handle leap years -- average Feb 28 and Mar 1
                        if tmin.shape[0] == 366:
                            #print("Leap year found "+str(yr))
                            leap = tmin[59:61,:,:]
                            #print("Leap shape:")
                            #print(leap.shape)
                            mean_leap = np.ma.mean(leap,axis=0)
                            #print("Mean_leap shape:")
                            #print(mean_leap.shape)
                            part1 = np.append(tmin[0:59,:,:],[mean_leap],axis=0)
                            tmin = np.append(part1,tmin[61:367,:,:],axis=0)
                            #print("New tmin shape:")
                            #print(tmin.shape)
                        tmin_array.append(tmin[:,:,:])
        tmin_mean = np.ma.mean(tmin_array,axis=0)
        tmin_std = np.ma.std(tmin_array,axis=0)
        tmin_min = np.ma.min(tmin_array,axis=0)
        tmin_max = np.ma.max(tmin_array,axis=0)
        
        # write netcdf files with results
        filename = "tmin_"+m+"_"+str(y)+"_10yr.nc"
        newfile = os.path.join(out_folder,filename)
        ncfile = netCDF4.Dataset(newfile,mode='w',format='NETCDF4_CLASSIC')
        lat_dim = ncfile.createDimension('lat', 48)     # latitude axis
        lon_dim = ncfile.createDimension('lon', 62)    # longitude axis
        time_dim = ncfile.createDimension('time', None) # unlimited axis (can be appended to).
        
        ncfile.title='Aggregate tmin values for WICCI downscaled climate data for '+m+' and 10-year window around year '+str(y)
        ncfile.subtitle="Eric Compas, compase@uww.edu"
        lat = ncfile.createVariable('lat', np.float64, ('lat',))
        lat.units = 'degrees_north'
        lat.long_name = 'latitude'
        lon = ncfile.createVariable('lon', np.float64, ('lon',))
        lon.units = 'degrees_east'
        lon.long_name = 'longitude'
        time = ncfile.createVariable('time', np.float64, ('time',))
        time.units = 'days since '+str(y)+'-01-01'
        time.long_name = 'time'
        
        temp_mean = ncfile.createVariable('tmin_mean',np.float32,('time','lat','lon')) # note: unlimited dimension is leftmost
        temp_mean.units = 'C' # degrees Celius
        temp_mean.standard_name = 'mean of minimum daily temperature'
        temp_mean.missing_value = -32768

        temp_std = ncfile.createVariable('tmin_std',np.float32,('time','lat','lon')) # note: unlimited dimension is leftmost
        temp_std.units = 'C' # degrees Celius
        temp_std.standard_name = 'standard deviation of minimum daily temperature'
        temp_std.missing_value = -32768

        temp_min = ncfile.createVariable('tmin_min',np.float32,('time','lat','lon')) # note: unlimited dimension is leftmost
        temp_min.units = 'C' # degrees Celius
        temp_min.standard_name = 'minimum of minimum daily temperature'
        temp_min.missing_value = -32768

        temp_max = ncfile.createVariable('tmin_max',np.float32,('time','lat','lon')) # note: unlimited dimension is leftmost
        temp_max.units = 'C' # degrees Celius
        temp_max.standard_name = 'minimum of minimum daily temperature'
        temp_max.missing_value = -32768
        
        # Write latitudes, longitudes
        # Note: the ":" is necessary in these "write" statements
        n_lat = n.variables['lat']
        n_lon = n.variables['lon']
        lat[:] = n_lat[:]
        lon[:] = n_lon[:]
        
        # write temp variable
        temp_mean[:,:,:] = tmin_mean  # Appends data along unlimited dimension
        temp_std[:,:,:] = tmin_std
        temp_min[:,:,:] = tmin_min
        temp_max[:,:,:] = tmin_max
        
        # write time
        n_time = n.variables['time']
        time[:] = n_time[:]
        
        # close file
        ncfile.close()
        
        # report progress
        print("Wrote: "+filename)

Processing base year 2010
Wrote: tmin_rcp45_2010_10yr.nc
Processing base year 2020
Wrote: tmin_rcp45_2020_10yr.nc
Processing base year 2030
Wrote: tmin_rcp45_2030_10yr.nc
Processing base year 2040
Wrote: tmin_rcp45_2040_10yr.nc
Processing base year 2050
Wrote: tmin_rcp45_2050_10yr.nc
Processing base year 2060
Wrote: tmin_rcp45_2060_10yr.nc
Processing base year 2070
Wrote: tmin_rcp45_2070_10yr.nc
Processing base year 2080
Wrote: tmin_rcp45_2080_10yr.nc
Processing base year 2090
Wrote: tmin_rcp45_2090_10yr.nc
Processing base year 2100
Wrote: tmin_rcp45_2100_10yr.nc
Processing base year 2010
Wrote: tmin_rcp85_2010_10yr.nc
Processing base year 2020
Wrote: tmin_rcp85_2020_10yr.nc
Processing base year 2030
Wrote: tmin_rcp85_2030_10yr.nc
Processing base year 2040
Wrote: tmin_rcp85_2040_10yr.nc
Processing base year 2050
Wrote: tmin_rcp85_2050_10yr.nc
Processing base year 2060
Wrote: tmin_rcp85_2060_10yr.nc
Processing base year 2070
Wrote: tmin_rcp85_2070_10yr.nc
Processing base year 2080
Wrote